### Pipeline for QnA with Memory

In [1]:
####Notes####
# chunked data length can be atmost 16 for azureopenai

In [2]:
# !pip3 install langchain
# !pip install openai
# !pip3 install openai chromadb
# !pip3 install tiktoken

In [3]:
# !pip3 install unstructured
# !pip3 install pdf2image
# !pip3 install pdfminer
# !pip3 install pdfminer.six
# !pip3 install pymupdf 

In [4]:
import os
import sys
import glob
import re
import importlib
import langchain
from langchain.document_loaders import WebBaseLoader, UnstructuredPDFLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import AzureOpenAI
from langchain.prompts import PromptTemplate
sys.path.append(os.path.join(os.getcwd(), '../scripts'))
import qna_memory_jn as qna_m
import constants as c
importlib.reload(qna_m)
importlib.reload(c)

<module 'constants' from '/Users/rachitjoshi/Documents/Finbot/notebooks/../scripts/constants.py'>

#### Environment

In [5]:
os.environ["OPENAI_API_KEY"] = "6cdb659e5a9d402e80c212fe8ea26483"

In [6]:
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
os.environ["OPENAI_API_BASE"] = "https://test-chatgpt-flomoney.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

#### Parameters

In [7]:
save_folder = '../data/pdfs/'
qna_prompt_template = os.path.join(os.getcwd(), '../scripts/qna_prompt_template.txt')
# prompt_template_file = None
condense_question_template = os.path.join(os.getcwd(), '../scripts/condense_question_template.txt')
# condense_question_template = None
pdf_list = [
    _ for _ in glob.glob(os.path.join(os.getcwd(), save_folder, '*.pdf'))
]
web_list = []

#### QnA Chain 

In [8]:
langchain_qna = qna_m.LangchainQnA(c.chunking_interface, c.embedding_model)

In [9]:
qna_chain = langchain_qna.main_function(
    pdf_list, web_list, qna_prompt_template, condense_question_template
)

/Users/rachitjoshi/finbot/lib/python3.9/site-packages/langchain/llms/openai.py:201: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/rachitjoshi/finbot/lib/python3.9/site-packages/langchain/llms/openai.py:786: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


#### QnA (using Chain)

In [12]:
question = 'okay then list me the pros and cons of investing in nvidia versus tesla'
result = qna_chain({"question": question})

[chain/start] [1:chain:ConversationalRetrievalChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:ConversationalRetrievalChain > 2:chain:LLMChain] Entering Chain run with input:
{
  "question": "okay then list me the pros and cons of investing in nvidia versus tesla",
  "chat_history": "\nHuman: so should i invest in tesla or nvidia?\nAssistant: As a financial advisor, I cannot provide specific investment advice without knowing your individual financial situation, goals, and risk tolerance. However, I can provide some information on both Tesla and Nvidia. Tesla has recently faced some challenges, including a decline in gross margins and an investigation by California's attorney general. On the other hand, Nvidia is seen as one of the biggest winners in the AI boom and has recently announced plans to expand its access to custom-made chips. It's important to conduct thorough research and consult with a financial professional before making any investment decisions."
}
[l

[llm/end] [1:chain:ConversationalRetrievalChain > 5:chain:StuffDocumentsChain > 6:chain:LLMChain > 7:llm:OpenAIChat] [7.46s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "As a financial advisor, it is important to conduct a thorough analysis of both Nvidia and Tesla before making any investment decisions. \n\nNvidia is a leader in AI chips and has been one of the biggest winners of the AI boom. The company has a strong focus on cost savings and free cash flow, which could be attractive to investors. However, the stock has already more than tripled in 2023 and is currently in a downtrend, so investors may want to wait for a pullback to key levels or a new base before investing. \n\nTesla, on the other hand, has been a monster stock over much of its history, especially during its stratospheric run from mid-2019 to late 2021. The company is betting big on the Cybertruck and autonomous vehicle technology, along with a possible tailwind from the Inflation 

In [13]:
print(result['answer'])

As a financial advisor, it is important to conduct a thorough analysis of both Nvidia and Tesla before making any investment decisions. 

Nvidia is a leader in AI chips and has been one of the biggest winners of the AI boom. The company has a strong focus on cost savings and free cash flow, which could be attractive to investors. However, the stock has already more than tripled in 2023 and is currently in a downtrend, so investors may want to wait for a pullback to key levels or a new base before investing. 

Tesla, on the other hand, has been a monster stock over much of its history, especially during its stratospheric run from mid-2019 to late 2021. The company is betting big on the Cybertruck and autonomous vehicle technology, along with a possible tailwind from the Inflation Reduction Act (IRA). However, Tesla is currently under investigation by California's attorney general regarding Autopilot safety and false advertising complaints, and the company's gross margins have been decli